In [1]:
import pandas as pd
import requests
import urllib.parse
import urllib.request
import sqlite3
from urllib.request import urlopen
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
# define the header
# write your own header here
# get your header at "chrome://version"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
# create engine
engine = create_engine('sqlite:///../../dataset/us/us_ticker_eps_annual.db')
# create database connection
database = '../../dataset/us/us_ticker_eps_annual.db'
conn = sqlite3.connect(database)
print('sql connect success!')
# import the list
ticker_list = pd.read_csv('ticker_list_with_name.csv')

sql connect success!


In [2]:
ticker_list = pd.read_csv('ticker_list_with_name.csv')

In [4]:
ticker_list[ticker_list.Symbol == 'MOBV']

,Symbol,Company Name
3673,MOBV,Mobiv-Acquisition


In [4]:
ticker_list['Company Name'] = ticker_list['Company Name'].str.replace(' ', '-')
# ticker_list.to_csv('ticker_list_with_name.csv',index=None)

In [5]:
ticker_list

,Symbol,Company Name
0,A,Agilent-Technologies
1,AA,Alcoa
2,AAC,Ares-Acquisition
3,AACG,ATA-Creativity-Global
4,AACI,Armada-Acquisition-I
...,...,...
6240,ZWS,Zurn-Water-Solutions
6241,ZY,Zymergen
6242,ZYME,Zymeworks
6243,ZYNE,Zynerba-Pharmaceuticals


In [6]:
ticker_symbol_list = ticker_list.Symbol.to_list()
ticker_name_list = ticker_list['Company Name'].to_list()
ticker_length = len(ticker_symbol_list)
# record result
log_list = []

In [ ]:

for i in range(ticker_length):
    try:
        html_data = requests.get('https://www.macrotrends.net/stocks/charts/{}/{}/eps-earnings-per-share-diluted'.format(ticker_symbol_list[i], ticker_name_list[i]),headers=headers)
        log_list.append('{} ok'.format(ticker_symbol_list[i]))
        print('{} ok'.format(ticker_symbol_list[i]))
    except:
        log_list.append('{} fail'.format(ticker_symbol_list[i]))
        
log_df = pd.DataFrame(log_list)
log_df

In [11]:
ticker_list[ticker_list.Symbol == 'AAPL']

,Symbol,Company Name
14,AAPL,Apple


In [14]:
html_data = requests.get('https://www.macrotrends.net/stocks/charts/{}/{}/eps-earnings-per-share-diluted'.format('AAPL', 'apple'),headers=headers)
soup = BeautifulSoup(html_data.text, 'lxml')
target_table = soup.find_all('table', attrs={'class':'historical_data_table'})
# Annual
target_table_01 = target_table[0]
ticker_eps_annual = pd.DataFrame(columns=['datetime','eps'])
for row in target_table_01.find_all('tr'):
    col = row.find_all('td')
    len_col = len(col)
    # print(len_col)
    # create the dataframe
    if len_col == 2:
        date = col[0].text
        eps = col[1].text[1:]
        temp = pd.DataFrame([[date, eps]],columns=['datetime','eps'])
        ticker_eps_annual = pd.concat([ticker_eps_annual,temp],ignore_index=True)

In [18]:
ticker_eps_annual.eps = ticker_eps_annual.eps.astype(float)
ticker_eps_annual.dtypes

datetime     object
eps         float64
dtype: object

In [19]:
target_table_02 = target_table[1]
ticker_eps_quarter = pd.DataFrame(columns=['datetime','eps'])
for row in target_table_02.find_all('tr'):
    col = row.find_all('td')
    len_col = len(col)
    # print(len_col)
    # create the dataframe
    if len_col == 2:
        date = col[0].text
        eps = col[1].text[1:]
        temp = pd.DataFrame([[date, eps]],columns=['datetime','eps'])
        ticker_eps_quarter = pd.concat([ticker_eps_quarter,temp],ignore_index=True)
ticker_eps_quarter.eps = ticker_eps_quarter.eps.astype(float)

In [57]:
# concat
datetime = 2020
eps = 2.02
temp = pd.DataFrame([[datetime, eps]],columns=['datetime','eps'])

In [24]:
ticker_eps_annual

,datetime,eps
0,2021,5.61
1,2020,3.28
2,2019,2.97
3,2018,2.98
4,2017,2.30
5,2016,2.08
6,2015,2.31
7,2014,1.61
8,2013,1.42
9,2012,1.58


In [30]:
try:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE {}'.format('AAPL'))
    conn.commit()
    print("{} table ceate successful!".format('AAPL'))
except:
    print("Table name already exists")
try:
    ticker_eps_annual.to_sql('AAPL', conn, if_exists='replace', index=False)
    print('{} data load success!'.format('AAPL'))
except:
    print('{} already exit'.format('AAPL'))

Table name already exists
AAPL data load success!


In [47]:
engine = create_engine('sqlite:///../../dataset/us/us_ticker_eps_quarter.db')
ticker_eps_quarter.to_sql('AAPL',con=engine)

54

In [ ]:
ticker_eps_quarter.to_sql('AAPL', con = engine)

In [42]:
from sqlalchemy import create_engine

#参数字段 sqlite:///<database path>
engine = create_engine('sqlite:///test.db')
import pandas
data = {
	'name':['小明','小红']
}

df = pandas.DataFrame(data)
df.to_sql('tb_test',con=engine) 

2